## Loading packages

In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('words')
from nltk.corpus import words
import re
import string
import datetime
from datetime import datetime
from dateutil.parser import parse

[nltk_data] Downloading package words to
[nltk_data]     /Users/taylorhonda/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Reading training data

In [ ]:
train = pd.read_csv("en_train.csv")
train.shape

(9918441, 5)

In [ ]:
train

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Brillantaisia,Brillantaisia
1,0,1,PLAIN,is,is
2,0,2,PLAIN,a,a
3,0,3,PLAIN,genus,genus
4,0,4,PLAIN,of,of
...,...,...,...,...,...
9918436,748065,12,PLAIN,of,of
9918437,748065,13,PLAIN,the,the
9918438,748065,14,ORDINAL,19th,nineteenth
9918439,748065,15,PLAIN,century,century


## Removing null values

In [ ]:
train[train["before"].isnull() == True]

,sentence_id,token_id,class,before,after
584464,46722,18,PLAIN,NaN,NaN
616107,49226,17,LETTERS,NaN,n a
684691,54634,1,PLAIN,NaN,NaN
747949,59607,5,PLAIN,NaN,NaN
965529,76612,7,PLAIN,NaN,NaN
...,...,...,...,...,...
9665672,729456,2,PLAIN,NaN,NaN
9665677,729456,7,PLAIN,NaN,NaN
9725113,733797,15,PLAIN,NaN,NaN
9864166,744065,15,PLAIN,NaN,NaN


In [ ]:
train[train["after"].isnull() == True]

,sentence_id,token_id,class,before,after
584464,46722,18,PLAIN,NaN,NaN
684691,54634,1,PLAIN,NaN,NaN
747949,59607,5,PLAIN,NaN,NaN
965529,76612,7,PLAIN,NaN,NaN
1347924,106058,6,PLAIN,NaN,NaN
...,...,...,...,...,...
9665672,729456,2,PLAIN,NaN,NaN
9665677,729456,7,PLAIN,NaN,NaN
9725113,733797,15,PLAIN,NaN,NaN
9864166,744065,15,PLAIN,NaN,NaN


In [ ]:
train.dropna(inplace=True)

In [ ]:
train = train[train['before'].map(lambda x: x.isascii())]
train = train[train['after'].map(lambda x: x.isascii())]
train.shape
# remove non-English rows
# https://stackoverflow.com/questions/65012603/removing-rows-contains-non-english-words-in-pandas-dataframe

(9852216, 5)

## Reading test data

In [ ]:
train[train["class"] == "DATE"]

,sentence_id,token_id,class,before,after
10,1,0,DATE,2006,two thousand six
51,3,7,DATE,2007,two thousand seven
80,5,0,DATE,2008,two thousand eight
111,8,1,DATE,4 March 2014,the fourth of march twenty fourteen
147,11,1,DATE,"April 10, 2013",april tenth twenty thirteen
...,...,...,...,...,...
9918259,748054,1,DATE,October 1865,october eighteen sixty five
9918267,748054,9,DATE,14 January 1867,the fourteenth of january eighteen sixty seven
9918276,748054,18,DATE,1866,eighteen sixty six
9918311,748056,8,DATE,1291,twelve ninety one


In [ ]:
test = pd.read_csv("en_test.csv")
test

,sentence_id,token_id,before
0,0,0,Another
1,0,1,religious
2,0,2,family
3,0,3,is
4,0,4,of
...,...,...,...
1088559,69999,13,close
1088560,69999,14,to
1088561,69999,15,machine
1088562,69999,16,languages


In [ ]:
test[test["before"].isnull() == True]

,sentence_id,token_id,before
83566,5382,3,NaN
149180,9602,1,NaN
261261,16795,8,NaN
261266,16795,13,NaN
310898,19971,8,NaN
378899,24332,10,NaN
421731,27088,2,NaN
478343,30770,4,NaN
652551,42023,11,NaN
751492,48355,1,NaN


In [ ]:
test.dropna(inplace=True)

In [ ]:
test["class"] = ""
test["after"] = ""

In [ ]:
test=test.reindex(columns= ['sentence_id', 'token_id', 'class','before', 'after'])
test.head()
test.shape

(1088551, 5)

In [ ]:
test = test[test['before'].map(lambda x: x.isascii())]
test.shape
# remove non-English rows
# https://stackoverflow.com/questions/65012603/removing-rows-contains-non-english-words-in-pandas-dataframe

(1081965, 5)

## Use Naive Bayes to classify `class`

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train["before"], train["class"], test_size=0.3)

In [ ]:
v = CountVectorizer(analyzer="char")
X_train_count = v.fit_transform(X_train.values)
X_train_count = X_train_count.toarray()
X_train_count[:1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
model = MultinomialNB()
model.fit(X_train_count, y_train.values.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
X_test_count = v.transform(X_test.values)
X_test_count = X_test_count.toarray()
y_pred = model.predict(X_test_count)
print(accuracy_score(y_test, y_pred))

0.9676424763970206


In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train_count, y_train)
# predictions = knn.predict(X_test_count)
# print(accuracy_score(y_test, predictions))

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# dtree = DecisionTreeClassifier()
# dtree.fit(X_train_count, y_train)
# predictions = dtree.predict(X_test_count)
# print(accuracy_score(y_test, predictions))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_estimators=10)
# rfc.fit(X_train_count, y_train.values.ravel())
# predictions = rfc.predict(X_test_count)
# print(accuracy_score(y_test, predictions))

In [ ]:
# # selecting columns for submission file
# test=test.reindex(columns= ['id', 'before', 'after'])
# test=test.iloc[:,:3]
# test.head()

## Use Neural Networks to classify `class`

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
import numpy as np
labelencoder_y_train = LabelEncoder()
y_train = labelencoder_y_train.fit_transform(y_train)
y_train = y_train.reshape(-1, 1)
print(np.max(y_train))

labelencoder_y_test = LabelEncoder()
y_test = labelencoder_y_test.fit_transform(y_test)
y_test = y_test.reshape(-1, 1)

print(y_train)
print(y_test)

15
[[11]
 [11]
 [11]
 ...
 [ 7]
 [ 2]
 [11]]
[[12]
 [11]
 [11]
 ...
 [12]
 [12]
 [11]]


In [ ]:
onehotencoder = OneHotEncoder(categories="auto")
y_train = onehotencoder.fit_transform(y_train).toarray()
y_test = onehotencoder.fit_transform(y_test).toarray()

In [ ]:
print(y_train.shape)

(6896551, 16)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
nn = Sequential()
nn.add(Dense(60, activation='relu')) # 10 features
nn.add(Dense(40, activation='relu')) # between
nn.add(Dense(16, activation='softmax'))

nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
nn.fit(X_train_count, y_train)

215518/215518 [==============================] - 167s 770us/step - loss: 0.0454 - accuracy: 0.9884


In [ ]:
# y_pred here refers to the 30% of training data
y_pred = nn.predict(X_test_count)
print(y_pred.shape)
y_pred = y_pred.argmax(axis=1)

print(y_pred.shape)
print(y_test.shape)
print(y_test)

(2955665, 16)
(2955665,)
(2955665,)
[12 11 11 ... 12 12 11]


In [ ]:
y_test = y_test.argmax(axis=1)
print(accuracy_score(y_pred, y_test))

0.9918289115985742


In [ ]:
print(confusion_matrix(y_pred, y_test))

[[     72       1       3       0       0       0       0       0       1
        0       0       0       0       0       0       0]
 [      9   38622     144      56    1325       0       6      39       5
        1     260      31       0     111       0       7]
 [     18    1325   77185       1     298       3      12       1      28
       26       0       0       0     157       4       5]
 [      0       0       1    2924       0       1       0       0       1
        5       0       0       0       0       3       0]
 [      0       0       0       0       0       0       0       0       0
        0       0       0       0       4       0       0]
 [      0       0       4       0       0    1486       0       2       6
        1       0       5       0       0       3       8]
 [      0       0      14       0       0       0     306       0       0
        0       0       0       0       0       0       0]
 [      0      67       3       0       0      18       0   34409    

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.53      0.68       135
           1       0.95      0.96      0.96     40238
           2       0.98      1.00      0.99     77386
           3       1.00      0.98      0.99      2991
           4       0.00      0.00      0.00      1623
           5       0.98      0.97      0.97      1538
           6       0.96      0.94      0.95       326
           7       0.87      0.76      0.81     45392
           8       0.98      0.98      0.98      4148
           9       0.98      0.97      0.98      1568
          10       0.99      0.88      0.93      3847
          11       0.99      1.00      1.00   2204457
          12       1.00      1.00      1.00    560757
          13       0.97      0.75      0.85      1170
          14       0.97      0.89      0.93       407
          15       0.71      0.87      0.79      9682

    accuracy                           0.99   2955665
   macro avg       0.89   

In [ ]:
X_test_count = v.transform(test["before"])
y_pred = nn.predict(X_test_count)
y_pred = y_pred.argmax(axis=1)
print(y_pred.shape)
print(y_test.shape)
print(y_test)

(1081965,)
(2955665,)
[12 11 11 ... 12 12 11]


In [ ]:
print(labelencoder_y_train.classes_)
print(labelencoder_y_test.classes_)

['ADDRESS' 'CARDINAL' 'DATE' 'DECIMAL' 'DIGIT' 'ELECTRONIC' 'FRACTION'
 'LETTERS' 'MEASURE' 'MONEY' 'ORDINAL' 'PLAIN' 'PUNCT' 'TELEPHONE' 'TIME'
 'VERBATIM']
['ADDRESS' 'CARDINAL' 'DATE' 'DECIMAL' 'DIGIT' 'ELECTRONIC' 'FRACTION'
 'LETTERS' 'MEASURE' 'MONEY' 'ORDINAL' 'PLAIN' 'PUNCT' 'TELEPHONE' 'TIME'
 'VERBATIM']


In [ ]:
y_pred = [labelencoder_y_train.classes_[y] for y in y_pred]
y_pred

['PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PUNCT',
 'CARDINAL',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PUNCT',
 'CARDINAL',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PUNCT',
 'CARDINAL',
 'PLAIN',
 'PUNCT',
 'VERBATIM',
 'PUNCT',
 'PUNCT',
 'DATE',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PUNCT',
 'MEASURE',
 'PUNCT',
 'VERBATIM',
 'PLAIN',
 'CARDINAL',
 'PLAIN',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PUNCT',
 'PUNCT',
 'PUNCT',
 'PLAIN',
 'CARDINAL',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'LETTERS',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PUNCT',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'CARDINAL',
 'PLAIN',
 'PLAIN',
 'PUNCT',
 'MEASURE',
 'PUNCT',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'PLAIN',
 'CARDINAL',
 'PLAIN',
 'PUNCT',
 'PUNCT',
 'PLAIN',
 'PLAIN'

In [ ]:
test["class"] = y_pred
test

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Another,
1,0,1,PLAIN,religious,
2,0,2,PLAIN,family,
3,0,3,PLAIN,is,
4,0,4,PLAIN,of,
...,...,...,...,...,...
1088559,69999,13,PLAIN,close,
1088560,69999,14,PLAIN,to,
1088561,69999,15,PLAIN,machine,
1088562,69999,16,PLAIN,languages,


In [ ]:
test.to_csv("en_test2.csv", index=False)
train.to_csv("en_train2.csv", index=False)